In [1]:
import spacy
import pandas as pd
import numpy as np
import neuralcoref
import ast
import random
from tqdm import tqdm_notebook

df1=pd.read_csv("scores1_freq.csv")
df2=pd.read_csv("scores2_freq.csv")
df1.head() #On vérifie que le fichier contient bien ce qu'il faut

/Users/ya/miniconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: spacy.morphology.Morphology size changed, may indicate binary incompatibility. Expected 104 from C header, got 112 from PyObject
  return f(*args, **kwds)
/Users/ya/miniconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: spacy.vocab.Vocab size changed, may indicate binary incompatibility. Expected 96 from C header, got 104 from PyObject
  return f(*args, **kwds)


,description,scores,freq
0,US-British actress Sienna Miller poses after u...,"{'US': 0.978723404, 'British': 0.978723404, 'S...","{'France': 0.16666666666666666, 'British': 0.1..."
1,People gather at a shopping mall in the Shatin...,"{'Shatin': 0.8297872340000001, 'Hong Kong': 1....","{'Hong Kong': 0.5, 'Shatin': 0.5}"
2,AFP presents a photo essay of 19 images by pho...,"{'AFP': 1.0, 'Christof Stache': 0.829787234000...","{'AFP': 0.25, 'Slug GERMANY-AGRICULTURE-CATTLE..."
3,A Palestinian worker harvests dates in the Jor...,"{'Palestinian': 0.489361702, 'Jordan Valley': ...","{'West Bank': 0.07142857142857142, ""the West B..."
4,Turkish Trade Minister Ruhsar Pekcan (R) and U...,"{'Turkish': 0.489361702, 'Ruhsar Pekcan': 0.93...","{'Commerce': 0.16666666666666666, 'Ruhsar Pekc..."


In [80]:
##Conversion des strings contenues dans le csv (colonnes score et fréquences) en dictionnaires

liste_scores1=[]
for string in df1.scores:
    liste_scores1.append(ast.literal_eval(string))   

liste_scores2=[]
for string in df2.scores:
    liste_scores2.append(ast.literal_eval(string))    

liste_freq=[]
for string in df1.freq:
    liste_freq.append(ast.literal_eval(string))

In [87]:
max(liste_scores1[0].values())
def normaliser_dict(l):
    for dico in l:
        if len(dico) != 0 :
            m= max(dico.values())
            if m >0 : # Possible que m =0 si description en Francais...
                for k in dico.keys():
                    dico[k] /= m 


In [89]:
normaliser_dict(liste_scores2)
normaliser_dict(liste_scores1)


In [64]:
for k in range(len(liste_scores2)):
    if len(liste_scores2[k]) == 0:
        print(k)
        for i in nlp(df1.description[k]).ents:
            print(i.label_, i)

2215
DATE April 10, 2017
WORK_OF_ART prison administration angry"
FAC the Fleury-Merogis prison
CARDINAL six
13389
DATE January 12, 2013
13407
DATE weekly
DATE January 9, 2013
FAC Paul VI
FAC Vatican
13423
DATE January 12, 2013
19373
FAC Bangui university
DATE January 4, 2013
20073
20368
20370
22186
23535
33239
DATE Christmas ornaments
FAC St. Peter's Cupola
FAC Vatican
DATE Christmas
DATE December 14, 2012
35217
DATE December 1, 2012
TIME earlier in the day
CARDINAL six
CARDINAL four
53808
FAC the Tokyo Tower
DATE 03 November 2003
DATE Christmas
CARDINAL sixty thousand
FAC the Tokyo Tower
DATE 03 November
64960
DATE 09 October 2003
82562
PRODUCT Congo</span


In [42]:
#Toute cette cellule est issue de l'autre notebook (création du csv) et permet donc d'avoir toutes les fonctions
#définies dans ce dernier à disposition.


#On importe le fichier contenant les scores de chaque fonction grammaticale
scoring = pd.read_csv('scoring.csv', delimiter = ";")
dict_val = {}

for i in range(48):
    dict_val[scoring['function'][i]] = scoring['score_norm'][i]

#On importe un modèle md pour avoir des mots vectorisés
nlp=spacy.load("en_core_web_md")  
neuralcoref.add_to_pipe(nlp,greedyness=0.5)

def dep_ent(ent, doc):
    """ Retourne la fonction grammaticale :  la 'dep', d'une entité. Cette fonction est nécessaire car elle permet d'affecter
    une dep à une entité composée de plusieurs mots ayant chacun une dep de base.
    Traite aussi le cas particulier des mots étant des conj ou des compound : leur vrai dep et celle du mot auxquels
    ils sont associés en tant que conj ou compound."""
    start= ent.start
    end=ent.end
    for k in range(start,end):
        if doc[k].head.text not in ent.text: 
            if doc[k].dep_=='conj':     
                tok=doc[k]            
                while tok.dep_=='conj':
                    tok=tok.head      
                return(tok.dep_)
            
            if doc[k].dep_=='compound':   
                tok=doc[k]            
                while tok.dep_=='compound':
                    tok=tok.head      
                return(tok.dep_)
            return(doc[k].dep_)    
    return doc[start].dep_

## RAJOUT DE FAC
def ent_good_type(ent): #filtre les entités selon leur type 
    return (ent.label_ == "PERSON"or ent.label_ == "NORP" or ent.label_ == "ORG" or ent.label_ == "GPE" or ent.label_ == "EVENT" or ent.label_ == "LOC" or ent.label_ == "FAC")

def sort_ent(doc):
    """Retourne la liste des entités en les filtrant selon leur type et en les triant de manière à avoir au début de
    la liste les entités ayant des coréférences."""
    ent_coref=[ent for ent in doc.ents if ent._.is_coref and ent_good_type(ent)]
    ent_vanilla=[ent for ent in doc.ents if ent_good_type(ent) and not ent._.is_coref]
    return ent_coref + ent_vanilla

def scores_doc(doc):
    """Retourne le score de chaque entité pour la méthode sans neuralcoref."""
    res={}
    for ent in doc.ents:
        if ent.text not in res.keys():
            res[ent.text]=dict_val[dep_ent(ent,doc)]
        else:
            res[ent.text]+=dict_val[dep_ent(ent,doc)]
    return res

def scores_doc_coref1(doc):
    """Retourne le score de chaque entité pour la méthode de base opérée sur le document resolved.
    Le document resolved est le document de base dans lequel toutes les références à un groupe de mot sont remplacées
    par celui-ci.
    Par exemple : My dad is home. He watches TV devient My dad is home. My dad watches TV.
    En raisonnant avec le nlp sur le document resolved, le nlp va détecter beaucoup plus de fois la même entité.
    Le principal inconvénient est que toutes les références sont remplacées, y compris celles qui ne sont pas associées 
    à des entités mais plutôt à des très longs bouts de phrase qui sont repris par un pronom comme "it"  """
    
    doc=nlp(doc._.coref_resolved)
    res={}
    for ent in doc.ents:
        if ent_good_type(ent):
            if ent.text not in res.keys():
                res[ent.text]=dict_val[dep_ent(ent,doc)]
            else:
                res[ent.text]+=dict_val[dep_ent(ent,doc)]
    return res

def is_in_cluster(ent,cluster):  #détermine si une entité est dans une des mentions d'un cluster
    for span in cluster.mentions:
        if ent.text in span.text:
            return True
    return False



def scores_doc_coref2(doc):
    """Cette méthode utilise neuralcoref mais au lieu d'agir sur le doc resolved, on va chercher a l'interieur des clusters
    associés au document. Pour rappel, un cluster contient un mot et ses références. Ce mot n'est pas forcément une entité.
    Un cluster est de la forme Trump : [Trump, he].
    Pour chaque entité, on va d'abord regarder si elle est coréférencée (donc mentionnée explicitement dans un cluster)
    ou pas.
    Si elle est coréférencée, on va chercher le cluster qui lui est associé.
    Une fois dans ce cluster, on va regarder pour chaque mention (réf à l'entité) si elle contient ou pas des entités.
    Si c'est le cas, on va alors indiquer que cette entité ne doit pas être retraitée par la suite car elle est traitée en
    tant qu'autre mention de l'entité que l'on traite actuellement.
    Par exemple, pour l'entité référencée Trump, on trouve le cluster suivant : Trump : [Trump, Donald Trump, he].
    Alors l'entité Donald Trump va être marquée comme traitée car elle est en fait équivalente à Trump.
    "he" n'étant pas une entité on a pas ce pb pour cette mention la.
    Ensuite on calcule le score de chaque mention (Trump, Donald Trump et he) en se basant sur leur fonction grammaticale.
    Puis on ajoute ces scores à celui de l'entité de base (Trump).
    
    Si l'entité de base n'est pas référencée, c'est un peu le même principe, la seule complexité en plus étant le fait que
    l'entité n'est pas forcément mentionnée telle quelle dans les mentions d'un cluster. 
    Par exemple, si notre entité de base est Sienna Miller, il est possible qu'on ait un cluster de la forme
    The british actress Sienna Miller : [The british actress Sienna Miller, she].
    Dans ce cas il faut rechercher si chaque mention contient l'entité de base avant de procéder comme pour les 
    entités référencées.
    
    """
    clusters=doc._.coref_clusters  #liste des clusters du doc. 
    res={} #le resultat sera un dictionnaire qui permet d'associer un score à chque entité
    ent_treated={} #dictionnaire pour différencier les entités traitées des autres
    ents_sorted=sort_ent(doc)  #on place les entités ayant une coref avant les autres
    for ent in ents_sorted:   
        ent_treated[ent.text]=0  #on initialise en affectant 0 à toutes les entités

    for ent in ents_sorted:     #pour chaque entité
        if ent_treated[ent.text]==0: #si elle n'est pas considérée comme "traitée"
            if ent._.is_coref:  #si elle est coréférencée (donc si elle apparait en tant que mention pour un cluster )
                #print(ent)
                for cluster in clusters:  #on va chercher le cluster associé à l'entité puisqu'elle est coref
                    if ent in cluster.mentions: # on regarde si l'entité est dans les mentions du cluster
                        for span in cluster.mentions: #Mtnt qu'on est dans le bon cluster, on regarde pour chaque span
                            if span.ents != [] : 
                                if ent.has_vector:
                                    max_sim=0
                                    max_span_ent=span.ents[0]
                                    for span_ent in span.ents: #on cherche l'entité dans le span la plus proche de ent
                                        if span_ent.has_vector:
                                            if ent.similarity(span_ent) > max_sim:
                                                max_span_ent=span_ent
                                                max_sim=ent.similarity(span_ent)
                                    ent_treated[max_span_ent.text]=1 #une fois qu'on l'a trouvé on la marque comme traitée
                            else:
                                max_span_ent=span        
                            if ent.text not in res.keys(): 
                                res[ent.text]=dict_val[dep_ent(max_span_ent,doc)]  #on affecte le score de l'entité
                            else:
                                res[ent.text] += dict_val[dep_ent(max_span_ent,doc)] 
            else: #si l'entité n'est pas exactement coréférencée par neuralcoref
                flag=0
                for cluster in clusters:
                    if is_in_cluster(ent,cluster) and ent.label_ != 'NORP': 
                        flag=1
                        for span in cluster.mentions:
                            #print(span)
                            if span.ents != [] :
                                if ent.has_vector:
                                    max_sim=0
                                    max_span_ent=span.ents[0]
                                    for span_ent in span.ents:
                                        if span_ent.has_vector:
                                            if ent.similarity(span_ent) > max_sim:
                                                max_span_ent=span_ent
                                                max_sim=ent.similarity(span_ent)
                                    ent_treated[max_span_ent.text]=1
                            else:
                                max_span_ent=span #on peut peut etre supprimer cette branche else et...
                            if ent.text not in res.keys():
                                res[ent.text]=dict_val[dep_ent(max_span_ent,doc)] #...replacer max_span_ent par span ici
                            else:
                                res[ent.text] += dict_val[dep_ent(max_span_ent,doc)]  #et ici
                if flag==0: #si l'entité n'est vraiment dans aucun cluster   
                    if ent.text not in res.keys():
                        res[ent.text]=dict_val[dep_ent(ent,doc)]
                    else:
                        res[ent.text]+=dict_val[dep_ent(ent,doc)] 
    return res


def freq_dict(L):
    n = len(L)
    L_unique = list(set(L))
    d = {}
    for a in L_unique:
        for b in L:
            if a == b:
                if a in d.keys():
                    d[a] += 1/n
                else:
                    d[a] = 1/n
    return(d)

In [136]:
##Les deux fonctions scores ci-dessous étaient les premières que nous avions écrites.

def score_sim1(doc,score_doc,k):
    """Prend en paramètre un doc, le fichier de scores associés à ce doc et un entier k correspondant à l'indice de la 
    description avec laquelle on veut comparer le doc. Retourne le score de similarité entre le doc et la description.
    Ici on calcule le score avec la méthode neuralcoref1 : on utilise liste_scores1."""
    res=0
    freq_doc=freq_dict([ent.text for ent in doc.ents])
    score_image=liste_scores1[k]
    freq_image=liste_freq[k]
    for i in score_doc.keys():
        if i in score_image.keys():
            res += (score_doc[i]+score_image[i])/2 #- abs(freq_doc[i]-freq_image[i])/2
    return res

def score_sim11(doc,score_doc,k):
    """Prend en paramètre un doc, le fichier de scores associés à ce doc et un entier k correspondant à l'indice de la 
    description avec laquelle on veut comparer le doc. Retourne le score de similarité entre le doc et la description.
    Ici on calcule le score avec la méthode neuralcoref1 : on utilise liste_scores1.
    Rajout d'une pénalité par entité différentes 
    """
    res=0
    freq_doc=freq_dict([ent.text for ent in doc.ents])
    score_image=liste_scores1[k]
    freq_image=liste_freq[k]
    for i in score_doc.keys():
        if i in score_image.keys():
            res += (score_doc[i]+score_image[i])/2 #- abs(freq_doc[i]-freq_image[i])/2
    score_image_tri = tri_dict(score_image)
    c = 0 
    for i in score_image_tri.keys():
        c+=1
        if i not in score_doc.keys():
            
            res -= 1/(c+1)**2
        
    return res

def score_sim2(doc,score_doc,k):
    """Prend en paramètre un doc, le fichier de scores associés à ce doc et un entier k correspondant à l'indice de la 
    description avec laquelle on veut comparer le doc. Retourne le score de similarité entre le doc et la description.
    Ici on calcule le score avec la méthode neuralcoref2 : on utilise liste_scores2."""
    res=0
    freq_doc=freq_dict([ent.text for ent in doc.ents])
    score_image=liste_scores2[k]
    freq_image=liste_freq[k]
    for i in score_doc.keys():
        if i in score_image.keys():
            res += (score_doc[i]+score_image[i])/2 - abs(freq_doc[i]-freq_image[i])/2
    return res


#Dernière fonction score développée. Elle donne de meilleurs résultats.

def score_sim3(doc,score_doc,k):
    """Prend en paramètre un doc, le fichier de scores associés à ce doc et un entier k correspondant à l'indice de la 
    description avec laquelle on veut comparer le doc. Retourne le score de similarité entre le doc et la description.
    Ici on calcule le score avec la méthode neuralcoref1 : on utilise liste_scores1.
    Par rapport aux fonctions précédentes, ici on a changé le calcul du score de similarité en "normalisant" les scores
    du doc et de l'image, au lieu de les diviser par 2 (ce qui était inutile)"""
    res=0
    score_image=liste_scores1[k]
    max_doc=max(score_doc.values())
    max_im=max(score_image.values())
    for i in score_doc.keys():
        if i in score_image.keys():
            if max_doc==0 or max_im==0:
                return 0
            res += score_doc[i]/max_doc +score_image[i]/max_im
    return res


def related_descr(doc):
    """Retourne la liste des indices correspondant aux descriptions ayant au moins 1 entité en commun avec le doc.
    Permet d'effectuer moins de comparaison dans la fonction best_image."""
    index_list=[]
    for k in range(len(liste_scores1)):
        for ent in doc.ents:
            if ent.text in liste_scores1[k].keys():
                index_list.append(k)
                break
    return index_list

def best_image1(doc):
    """Retourne l'indice de l'image dont la description correspond le plus au doc passé en paramètre.
    La correspondance (matching) se fait avec score_sim1."""
    best_score=0
    best_descr=0
    score_doc=scores_doc_coref1(doc)
    for k in related_descr(doc):
        if score_sim1(doc,score_doc,k) > best_score:
            best_score=score_sim1(doc,score_doc,k)
            best_descr=k
    return best_descr



def best_image11(doc):
    """Retourne l'indice de l'image dont la description correspond le plus au doc passé en paramètre.
    La correspondance (matching) se fait avec score_sim1."""
    best_score=0
    best_descr=0
    score_doc=scores_doc_coref1(doc)
    for k in related_descr(doc):
        if score_sim11(doc,score_doc,k) > best_score:
            best_score=score_sim1(doc,score_doc,k)
            best_descr=k
    return best_descr

def best_image2(doc):
    """Retourne l'indice de l'image dont la description correspond le plus au doc passé en paramètre.
    La correspondance (matching) se fait avec score_sim2."""
    score_doc=scores_doc_coref2(doc)
    best_score=0
    best_descr=0
    for k in related_descr(doc):
        if score_sim2(doc,score_doc,k) > best_score:
            best_score=score_sim2(doc,score_doc,k)
            best_descr=k
    return best_descr

def best_image3(doc):
    """Retourne l'indice de l'image dont la description correspond le plus au doc passé en paramètre.
    La correspondance (matching) se fait avec score_sim3."""
    score_doc=scores_doc_coref1(doc)
    best_score=0
    best_descr=0
    for k in related_descr(doc):
        if score_sim3(doc,score_doc,k) > best_score:
            best_score=score_sim3(doc,score_doc,k)
            best_descr=k
    return best_descr

In [138]:
%%time

#On crée une liste d'indices associés à 100 descriptions différentes de la base.

rand_ind_list=[]
for i in range(500):
    num = random.randint(0, df1.shape[0])
    while(num in rand_ind_list):
        num = random.randint(0, df1.shape[0])
    rand_ind_list.append(num)
print(rand_ind_list)

#On calcule le pourcentage de descriptions i telles que la description la plus proche de i dans la base soit i
#On a constaté que souvent, lorsqu'une description ne matche pas avec elle-même, elle matche avec une description très
#voisine qui est probablement proche de la description initiale en terme d'indice. En effet, on a souvent par exemple une
#dizaine de photos illustrant le même évènement/fait d'actualité.
#count2 doit remédier en partie à ce pb en étant incrémenté si la description la plus proche de i est i ou est
#proche de i en terme d'indice (entre i-50 et i+50).
count=0
count2=0
rand_int_list=range(100)
for i in tqdm_notebook(rand_ind_list):
    best=best_image1(nlp(df1.description[i]))
    flag=0
    if best==i:
        count+=1
        flag=1
    if best in range(i-50,i+50):
            count2+=1
            flag=1
            
    if flag==0:
        print(i,best) #si aucun compteur n'est incrémenté dans l'itération, potentielle erreur de l'algorithme.
        #On print alors i et best pour voir par la suite si l'erreur était une vraie erreur ou pas.
print(count,'% et ', count2,'%')

[47754, 73499, 32126, 73887, 28290, 6959, 351, 77726, 80471, 10175, 46824, 16292, 67029, 73858, 49191, 50446, 13245, 69625, 71936, 13389, 64440, 63032, 68882, 31006, 43915, 10493, 79435, 19707, 37056, 38230, 67279, 35198, 26349, 82338, 32364, 15461, 48023, 38638, 78894, 1838, 19725, 57180, 44592, 7096, 17126, 26051, 44033, 38088, 5466, 5701, 39072, 39650, 28237, 59294, 25738, 2762, 42730, 50912, 18024, 6753, 55510, 44698, 1399, 68504, 27855, 32905, 8751, 77024, 14961, 16970, 52980, 52278, 33352, 59959, 39934, 7324, 19834, 21660, 31139, 3535, 15431, 68446, 50197, 41370, 49937, 56831, 49723, 48206, 74853, 34770, 17304, 49930, 45924, 22828, 63948, 46779, 18293, 79232, 44, 63185, 24537, 35325, 54775, 21243, 39362, 814, 26775, 76236, 17519, 58975, 8275, 15404, 55793, 24593, 14691, 64591, 71803, 74845, 77000, 5333, 62548, 25818, 73305, 6336, 40145, 77400, 21252, 55534, 52064, 68972, 62574, 49957, 56497, 81152, 77523, 47104, 66757, 57725, 55458, 6417, 71051, 29823, 56111, 57331, 55352, 73229,

/Users/ya/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


32126 30457
351 53283
13389 0
79435 65499
15461 14768
78894 79430
19725 21449
5466 4458
59294 59103
25738 25618
2762 603
50912 49355
59959 60014
19834 19534
31139 31074
15431 14565
49723 47644
63185 64417
39362 38126
8275 6585
14691 37671
71803 70721
62548 47158
47104 46094
38828 39433
78016 78215
11998 12072
56010 62332
57269 71585
11615 10997
16513 22309
9892 9813
62968 63277
75516 74587
21343 21398
14999 12457
50224 50852
32595 11652
18345 18214
24432 23600
38331 33652
6520 6458
30414 30529
13751 13404
2775 603
9416 31216
2184 462
18378 18448
80858 80774
25183 25109
82066 81961
68974 69041
26510 26370
14784 18646
16535 16715
30261 30349
7224 7045
24718 24440
74192 64843
46296 4878
32712 34035
45756 12340
32239 30457
39710 40109

303 % et  436 %
CPU times: user 23min 56s, sys: 10 s, total: 24min 6s
Wall time: 24min 33s


In [94]:
df1.description[68747]

'A resident bleeds after police and local government security beat him during an an eviction of some 100 properties in Jakarta, 02 October 2003. Hundreds of residents of 15,6 hectare slum area are protesting after they lived for years on the land. Jakarta with a population estimated at between eight and 10 million, is desperately short of low-cost housing and other amenities. Millions have no access to proper sewerage or piped water. AFP PHOTO/Bay ISMOYO'

In [139]:
%%time

count=0
count2=0
rand_int_list=range(100)
for i in tqdm_notebook(rand_ind_list):
    best=best_image11(nlp(df1.description[i]))
    flag=0
    if best==i:
        count+=1
        flag=1
    if best in range(i-50,i+50):
        count2+=1
        flag=1
        
    if flag==0:
        print(i,best) #si aucun compteur n'est incrémenté dans l'itération, potentielle erreur de l'algorithme.
        #On print alors i et best pour voir par la suite si l'erreur était une vraie erreur ou pas.
print(count,'% et ', count2,'%')

/Users/ya/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


32126 30457
351 53283
13389 0
79435 65499
15461 12858
48023 48084
78894 79430
19725 21449
5466 4458
59294 59103
25738 25618
2762 603
50912 49355
59959 59807
19834 19534
31139 31074
15431 14565
63185 64417
39362 38126
8275 6565
14691 37671
71803 70721
62548 67797
47104 46094
38828 39433
78016 78168
11998 12072
57269 71585
11615 10997
9892 9813
62968 62528
75516 74587
21343 21398
14999 12457
50224 50852
32595 11652
18345 18214
24432 23600
38331 33652
6520 6458
30414 30529
13751 13404
2775 603
9416 31216
2184 462
18378 18448
80858 80774
25183 25109
82066 81961
26510 26370
14784 18646
16535 16715
30261 30349
7224 7045
24718 24440
74192 64843
46296 4878
75123 74690
32239 30457
39710 40109

310 % et  440 %
CPU times: user 24min 10s, sys: 9.98 s, total: 24min 20s
Wall time: 24min 59s


In [30]:
print(df1.description[1625],'\n',df1.description[1515])
#On voit par exemple ici que malgré plus de 50 indices d'écarts, les descriptions sont quasiment identiques.
#Le pourcentage de réussite de l'algorithme doit alors être encore revu à la hausse.
#Après avoir étudié les descriptions associées aux indices ayant été print à la cellule précédente, on peut en fait
#estimer que le vrai pourcentage de réussite de l'algorithme est supérieur à 90%.
#Les seules erreurs constatées sont liées à  des descriptions identiques pour des dates différentes (on ne prend
#actuellement pas en compte les dates dans les entités) ou bien des descriptions très proches avec seulement la 1ère phrase
#qui diffère. Il faudrait alors accorder plus d'importance à la 1ère phrase.

Colombian President Juan Manuel Santos (L) looks the FIFA World Cup trophy next to France's 1998 World Cup Champion, French-Argentine former footballer David Trezeguet, in Bogota, Colombia, on April 3, 2018 during the FIFA Trophy World Tour ahead of the World Cup to be held in Rusia between June 14 and July 15.   
 Colombian President Juan Manuel Santos (L) holds the FIFA World Cup trophy next to France's 1998 World Cup Champion, French-Argentine former footballer David Trezeguet, in Bogota, Colombia, on April 3, 2018 during the FIFA Trophy World Tour ahead of the World Cup to be held in Rusia between June 14 and July 15.  


In [115]:
d= liste_scores1[10].copy()
d

{'Britain': 0.20930232536533258,
 'State for Exiting the European Union': 0.45348837208031373,
 'Brexit': 0.7325581395984315,
 'Stephen Barclay': 0.5465116279196863,
 'London': 0.45348837208031373,
 'British': 0.5348837208031368,
 'Boris Johnson': 1.0,
 'Brussels': 0.45348837208031373}

In [123]:
def tri_dict(d): 
    """Tri décroissant """

    d_sort = {}
    for k,v in sorted(d.items(), key =lambda x :x[1], reverse= True):
        d_sort[k]=v
    return d_sort
    

## Essai avec un arctile parlant d'une invention pour aider les gens ayant des problèmes pour communiquer. 


In [142]:
doc = "Last November, Joe Morris, a 31-year-old film-maker from London, noticed a sore spot on his tongue. He figured he’d bitten himself in his sleep and thought nothing more about it until halfway through the winter holidays, when he realised the sore was still with him. He Googled “cut on tongue won’t heal” and, after sifting through pages of medical information on oral cancer, he decided to call his doctor. The cut was nothing, Joe was sure: he was a non-smoker with no family history of cancer. But he’d make an appointment, just in caseI’m sure it’s nothing, the doctor said. You’re not a smoker, and you’re 31 years old. But see a specialist, just in case.I’m sure it’s nothing, the specialist said, you don’t check any of the boxes, but we’ll do a biopsy, just in case.When the biopsy results came back positive for cancerous cells, the specialist said that the lab must have made a mistake. The second time Joe’s biopsy results came back positive, the specialist was startled. Now Joe was transferred to Guy’s hospital, which has one of the best oral cancer teams in Britain.The oncologists at Guy’s reassured Joe again: the cancerous spot was small, and cancer of the tongue typically starts on the surface and grows inward. This tiny sore could likely be nipped out without much damage to the rest of his tongue. They’d take an MRI to make sure there wasn’t any serious inward growth, and then schedule the surgery.The image revealed a tumour like an iceberg. It was rooted deep in the base of Joe’s tongue, mounding upward and out, its tip breaking the surface just where the telltale sore was located. “When the doctor told me the news, there was a work email that was bugging me and I still had that on my mind,” Joe wrote to me last summer. “As he was explaining to me that I was going to lose my tongue, I was redrafting a reply in my head.”“You’re going to lose two-thirds of your tongue,” the doctor was telling him. “This is going to seriously affect your ability to eat. And your speech.”Joe wanted to know how the surgery would affect his speaking. Would he have a lisp?The doctor hesitated, and then looked at his hands. “Your family will still be able to understand you.”A week before the surgery, Joe started to panic: he realised that he might never speak again. Even if he did, he would no longer sound like himself. Knowing that he was about to lose a huge part of his identity, Joe asked a friend to film an interview with him so that he would have a permanent record of his voice.In the video, Joe’s speech is already beginning to falter: he has a bit of a lisp, and he has to sip water frequently and take breaks to withstand the strain of talking. He is dressed in a black knitted V-neck sweater, and seated near a window through which you can see the London skyline at dusk. He is pale, with sunken, blue eyes, dark, shaggy hair and three days of stubble. He looks a little unwell, a little sad, and a little rueful, as if he’s unsure about being the centre of attention. He keeps ducking his head and looking away, or making jokes. When asked to state the date, he smirks and says, with wry formality, “The date is, I believe, the 24th of February, the year of our Lord 2017.”Speaking to the camera, Joe struggles genially to articulate what it feels like to contemplate losing his voice for ever. “I’m not what you’d call a vain man,” he says, quietly. “Usually it’s very far into the day before I’ve looked in the mirror. I don’t care about any of that.” He takes a moment. “But I am human. And the idea that I’m going to not look like me or sound like me … terrifying.” He swallows. “And also my job, my life, is all about communication, all about talking. I love talking,” he says feelingly, with a little smile. “I’ve got a few things to say.”Shortly before this video was taken, the friend behind the camera had come to Joe with some news. He had found a company outside Boston called VocaliD, which creates custom digitised voices for people who use devices to help them speak. The company could use recordings of Joe speaking to recreate his own voice on a computer for him to keep and use for ever.When they contacted VocaliD’s founder, a speech pathologist named Rupal Patel, she explained that it would be possible to digitally reconstruct Joe’s voice if, before his surgery, he was able to “bank” his voice. This meant recording the few thousand sentences that VocaliD has developed to capture all the phonemes in the English language.Joe agreed to try. He recorded several hundred sentences and then, realising the magnitude of the task, stopped for several days. “This was my last week of freedom and I had a lot of stuff to do, people to see, life to live (and steaks to eat),” he wrote to me. Two days before the surgery, he started again. Banking his voice was slow and painful – by then, it was excruciating to talk, and he was trying to be at his most eloquent. On the final day, he recorded late into the night.The next morning, Joe went back to hospital and had his tongue cut out, joining the ranks of those who cannot, in any traditional sense, speak.There are surprisingly many ways for the power of speech to fail. There are disorders such as stuttering or apraxia, in which syllables are scrambled; motor neurone disease and cerebral palsy, which rob people of the muscle control required to articulate; traumatic brain injury; stroke; anatomical excisions like Joe’s; multiple sclerosis; autism. In the US, more than 2 million people require digital “adaptive alternative communication” (AAC) methods to help compensate for speech deficits. A 2008 study by the disability charity Scope estimated that 1% of people in Britain use or need AAC.Modern adaptive alternative communication often involves the type of device made famous by Stephen Hawking – a small computer or tablet that plays aloud words typed into it. Before the invention of the first modern text-to-speech communication device in 1969, people with muscular or vocal disorders had to use “sip-and-puff” typewriters, which were operated by inhaling and exhaling through a straw. By 1986, when Hawking began using a speech device, AAC technology had improved significantly. The programme he used, known as the Equalizer, allowed him to press a switch to select words or phrases on a desktop computer and, later, a smaller computer mounted to his wheelchair.The 2014 biopic of Hawking’s life, The Theory of Everything, contains a stark reminder of the loss that this technology tries to amend. When Hawking and his first wife, Jane, first hear what will be Hawking’s new voice, they are stunned. After a moment of speechlessness, Jane offers a timid objection: “It’s American.” The moment is played for laughs, but it marks a trauma. Our voices are coded with information by which others know us – age, gender, nationality, hometown, personality, mood – but they are also coded with the information by which we know ourselves. When your voice is no longer English, what part of your Englishness do you lose?Hawking’s case is one of the most striking examples of the way a person’s voice shapes their identity. Though the robotic quality of his digital voice (and the American accent) felt inappropriate at first, it came to be his trademark. Hawking reshaped himself around his new voice, and years later, when he was offered the opportunity to use a new voice that was smoother, more human-sounding, and English, he refused. This felt like “him” now.The “Stephen Hawking voice” doesn’t belong only to Hawking. In the years since it was created, the same voice has also been used by little girls, old men, and people of every racial and ethnic background. This is one of the stranger features of the world of people who rely on AAC: millions of them share a limited number of voices. While there is more variety now than before, only a few dozen options are widely available, and most of them are adult and male.“Walk into a classroom of children with voice disorders and you’ll hear the exact same voice all around you,” Rupal Patel of VocaliD told me. Ten years ago, she was at a speech disorders conference when she came upon a little girl and a man in his mid-50s who were using their devices to have a conversation. They were speaking in the same adult, male voice. Patel was horrified. “This is just continuing to dehumanise people who already don’t have a voice to talk,” she told me.The film critic Roger Ebert, whose jaw was removed to treat cancer, wrote in 2009 about how frustrating it was to use one of these generic voices: “I sound like Robby the Robot. Eloquence and intonation are impossible.” He was tired of being ignored in conversations or coming across like “the village idiot”. He went on: “We put men on the moon, people like to say about such desires: Why can’t I have a voice of my own?”This is the problem Patel has set out to solve. In 2007, she began researching technology that would allow her to make customised digital voices that sounded more like the humans they would represent. By 2014, the technology was sufficiently developed for Patel and her team to set up what they claim is the world’s first “voice bank”, an online platform where anyone with an internet connection can “donate” their voice by recording themselves reading aloud on to the VocaliD Voicebank, which is programmed with stories crafted to capture all the phonemes in the English language. (Early donors were required to upload 3,487 sentences; now, Geoff Meltzner, VocaliD’s Director of Research, can create a voice with as few as 1,000 sentences, though more material makes for a more human-sounding voice.)Each donation is catalogued in a library of voices that VocaliD can then use when crafting a new voice for a client. The company offers clients “BeSpoke” voices – custom-made voices that combine the sound of a client’s own voice with the vocabulary supplied by a donor. This way, a teenager could use his brother’s donated voice, or a perfect stranger’s from the Voicebank, whichever is the closest approximation to their imagined vocal quality. (Clients like Joe bank their voices for a purpose VocaliD calls “voice legacy”: they record themselves for later, and then, when the time comes, are given back a digital file of their own voice.)Creating a new digital voice like this requires splitting two elements of the human voice that normally function as one: the source and the filter. “Source” is the term for the vocal cords, larynx and throat muscles – the part of the body responsible for making sound when we laugh, yell or talk. As Geoff Meltzner, VocaliD’s vice-president of research, explained, your source is like a fingerprint. “There’s enough identity in each source alone to make it unique among all other sources.” The voice’s “filter” is the muscles (tongue, lips, pharynx etc) that shape those sounds into discrete, discernible words.VocaliD’s technology works by capturing a few seconds of vowel sound (the source) from the recipient, and applying it to the filter provided by a donor. The combination allows the production of a voice that’s largely “of” the recipient. By tweaking his algorithms, Meltzner can offer voice that is “warmer” (more nasal) or more “authoritative” (lower pitch) or “brighter” (full of high overtones).When a new voice is completed, it is added as a plug-in to whatever speech device its owner already uses. Recently, VocaliD added a feature on their own app that allows clients to adjust their voice to get exactly the timbre and volume they want. The system is designed to be convenient, but it still occasionally falls victim to glitches. One time, a teenage client called Patel in a panic because she had updated the software on her iPhone and lost her voice.Donating your voice – unlike, say, a kidney – usually takes a few days, and you’re awake for all of it. There’s no screening process and no equipment involved except a laptop and an internet connection. One lazy day last winter, I decided to donate my voice from bed, which is how I found myself pitched forward with my laptop, mouth to the built-in microphone, insisting: “That tiramisu is to die for! That tiramisu is to die for!”VocaliD’s Human Voicebank runs on an internet browser and is set up to look slightly like a video game: an indigo-blue backdrop frames a jaunty cartoon mouth with arms and legs standing next to a line of text, which you read aloud. Once you’re satisfied with your delivery, you click to upload the sentence to the bank, and a new line emerges. A bar at the bottom of the screen tracks your progress.There are so many sentences to read that people usually spread their donation out over days and weeks, doing just a few hours at a time. In an attempt to make the long exercise entertaining, VocaliD gives options to read material according to your interests: poetry, say, or science fiction. The sentences I read ranged from the proverbial (“It ain’t over till the fat lady sings!”) to the banal (“Did you see it on Twitter?”) to the sobering (“This is an emergency. Get help now”). Some felt too private. Donating any part of your body is intimate. It strikes at the core of something we appreciate about ourselves: we are each single-editions. The voice is perhaps a uniquely personal gift. It’s both physical and metaphysical. It’s the emissary between our corporeal selves and the rest of the world.When “I love you” came up, I re-recorded it over and over in a slight panic. (Other people, I’ve since learned, come upon this line and burst into tears.) What kind of I-love-you was this? Should I be speaking as if to a lover, a parent, or a pet? Was this I-love-you an assurance of the strength of the feeling (“I love you”) or its target (“I love you!”)? Should it have the butterflies tone of a first I-love-you, like a shy declaration, or of warm confirmation, like a mother saying goodnight to her child? Now sweating slightly, I recorded the phrase in what I hoped was a warm, neutral tone, deemed it not too stiff on playback, squeezed my eyes shut, and clicked. It was my last donation of the day.Shortly after making my donation, I visited Rupal Patel at VocaliD’s offices in the western suburbs of Boston. Patel is slight and energetic, with bright eyes, a pert, chin-length bob, and brilliant enunciation. She is ardent about how miraculous a personalised voice can be for someone who has been unvoiced. When disabled people have communication impairments, she explained, it increases the likelihood that they’ll be removed from the workforce, isolated socially, mistakenly identified as cognitively impaired, or rendered invisible.Humans respond with special attention and empathy to other human voices, and to unconsciously equate the ability to speak with presence of mind. In 2010, medical anthropologist Mary Wickenden wrote a thesis on teenage AAC users titled Teenage Worlds, Different Voices, in which she pointed out: “If you cannot talk, it may be hard to prove that you [think] … language expressed out loud makes our subjectivity ‘more real’.”Those who cannot speak are constantly reminded of their “unreality” in the eyes of society. Of the seven voices VocaliD completed in its first year, six were for children or teenagers with cerebral palsy, many of whom complained that strangers tend to either ignore them entirely, directing any questions or conversation to their parents, or speak to them as though they were toddlers.Type-to-talk technology varies widely based on the needs of the individual user: people who have muscular command of their fingers can simply type on to a traditional keyboard and hear the words echoed out over a speaker. More common is a version in which the user scrolls and selects from a selection of words, phrases or symbols on screen, using a lever or switch positioned closest to the limb they have best control over. For those who can’t use levers, there are screens that track eye movement and are programmed to read aloud a phrase or symbol when the user has stared at it long enough.Even for people who become fluent users of type-to-talk technology, the devices can be frustrating. Often, you have to wait for a cursor to pass over more than a dozen letters or symbols before it arrives at the one you need – and if you miss, you have to wait for the cursor to cycle all the way through again. Until recently, many devices didn’t even come with words or symbols for female genitalia, leaving no easy way to talk candidly with a friend or partner about sex, or to alert a caregiver about, say, a urinary tract infection, or abuse.Preprogrammed voices are often inappropriate to their user’s age, or frustratingly robotic. Patel told me about one of her clients, a teenager named Sara Young, for whom they were building a new voice. At the time, Sara was using the same voice (“Heather”) as her mother’s GPS and some bank ATMs. Several girls with speech disabilities in Sara’s class at school use Heather, which means that in group settings it’s nearly impossible to distinguish who is saying what unless you’re looking closely. Like lots of her peers, Sara often played around with the different preprogrammed voices in her device – trying on different options for a day or two, or speaking in an adult male voice to give herself a laugh – but still she was frustrated. When I visited the office, Patel and Meltzner were putting the finishing touches on Sara’s BeSpoke voice, which they were constructing using a few “ahhh” sounds that Sara had recorded, and a donated voice. They were hoping for it to be ready by Christmas.On my second day with Patel, I accompanied her to a technology fair at the Cotting School in Lexington, Massachusetts, a private school for special-needs students, several of whom are VocaliD clients. The company often does outreach at schools, both to offer their products to children using AAC, and to recruit new donors – they are always short on young donor voices. The fair was full of parents and children with cerebral palsy, including Sara. Like many kids with cerebral palsy, Sara is exceptionally small for her age, because eating requires muscular control she doesn’t always have. Her hair is dark and wavy and dyed with teal streaks, and when we met, she was wearing a light pink, long-sleeved shirt, the bag affixed to her motorised wheelchair was pink, and the foot she uses to drive (her only limb with reliable fine motor control) was shod in a pink sneaker.As is often the case for people with movement and muscle disorders, Sara’s body moves in spasms. Her tongue flexes in and out of her mouth, and her neck twists from side to side. Her arms curl and unfurl like leaves. She can’t eat or shower or use the bathroom without help. She uses silicon straws to drink, because she bites uncontrollably when she sucks, which destroys regular straws. (Before her parents figured out about the silicon, they cut short lengths of fish-tank tubing for her.) She uses her left foot to do her homework assignments on an iPad and – with the help of duct tape and markers – to draw. When she speaks, it’s through an AAC device mounted on her roller chair, which senses her eye movements as a substitute for typing.Her bodily presence, which seemed at first glance childlike, belied her personality, which was classic teen. She kept her wheelchair still, occasionally shuffling back and forth a bit listlessly, like someone rocking back and forth on her heels. When she got bored, she made a little loop. She has a blue-and-pink nose ring, and she hates having to carry an outmoded cell phone. (“BLACKBERRY SUCKS,” she told me.) She has ferocious eyebrows and sharp, dark, funny eyes, which she rolls frequently.Because she is a strong communicator, Sara has become an occasional ambassador for the other kids in her AAC community. At this technology fair, Sara and her mother, Amy Young, took the stage to give the keynote address. Sara spoke first, giving a few sentences of introduction she had written in advance on her device. Her voice did indeed sound just like an ATM. “HELLO EVERYONE, MY NAME IS SARA. I AM 16. WHEN I DON’T HAVE A [DEVICE], PEOPLE BABY TALK TO ME OR JUST TALK TO MY MOM. SOMETIMES I AM SLOW TO SPEAK AND SO THEY JUST TALK OVER ME. THEY DON’T KNOW HOW TO WAIT FOR MY ANSWER.”The truth of this became clear later in the day, when Amy and Sara conducted a joint Q&A. When Sara was asked what she uses her iPad for, she began staring at her screen with particular focus, twisting her head to keep her gaze levelled at the device regardless of her neck spasms. Thirty seconds passed, then 60. Everyone sat in silence, looking at her. Ninety seconds later, the computer spoke in a fluid stream: “HWFACEBOOKIGSNAPCHATMUSIC.”Amy translated. “Homework, Facebook, Instagram, Snapchat and music.”In the hour-long Q&A, Sara spoke fewer than 30 words. As is their custom, Amy did the majority of the speaking, partly for the sake of time and partly because Sara routinely relies on her mother to understand and translate her non-verbal cues. “It’s a tremendous amount of energy for her,” Amy explained to me later. “And while we encourage people to talk to her directly, sometimes she’ll reply by looking at me, like, ‘Can you just answer that?’”Sara has a canny sense of humour, but her speaking style and pace lends itself more to well-placed interjection. In the middle of Amy’s careful explanation of why Apple’s bluetooth systems are incompatible with the motorised wheelchair, Sara cut in to phrase things more succinctly: “IDIOTS.” She peppered her mother’s sentences with brief exclamations in her own voice: “Yeah!” When she was out of the spotlight and speaking to people who knew how to adapt to her ways of speaking, conversation flowed more naturally. After the Q&A, scrolling through Instagram with a young aide at her high school, Sara let out a series of amused hoots. The aide shook her head in mock disapproval at the screen and said conspiratorially: “Your class is crazy.”Sara laughed. “YOU DON’T EVEN KNOW.”The disconnect between the spirit of Sara’s words and the robotic deadpan of their delivery was jarring. “The digital voice gets kind of lost,” said Amy. “When we heard about VocaliD, we thought: ‘How cool it would be to create something a little more natural.’ Sara hasn’t had the experience of having her voice change as she ages, and so that would be neat, too. If the voice is that much more natural, I’m hoping it won’t get lost as much.”When I told Patel about this conversation, her eyes sparkled. “I really want people to be able not just to hear Sara, but in hearing her, to see her and experience her. When she belts out that “Yeah!” or that “No!” or whatever she says with her natural voice and then transitions to her device, boy wouldn’t it be nice if those two types of communication felt fluid. In an ideal world, she’d never have to use that thing. She’d be wearing a pair of glasses and an oculus and she’d have all these messages that she’d typed out. It wouldn’t be stigmatising. She wouldn’t have to be seen as an “other” communicator. That’s the wave of the future.”For Joe, the transition from being able-bodied and seamlessly verbal to someone who seems physically – and, to the careless observer, mentally – disabled has been astonishing, and deeply painful. When he woke up from the surgery, it was the first time he was really, truly speechless. The doctors had excised most of his tongue – “and you have to remember that a large portion of your tongue you can’t see, because it’s in your throat,” Joe reminded me – and then took a long strip of his quadricep and attached the muscle to the flap that was left in his throat. They hoped that in time he would gain enough control of the new muscle to swallow and, in time, form words.For the first week and a half, the tracheotomy tube diverted all air from his windpipe out through his neck, so that even if he tried to speak, no sound would come. “I felt completely trapped, a prisoner in my own body,” he told me via email. He could write things down to let the medical staff know if he was hungry or in pain. “But in terms of meaningful communication, you’re locked out completely.” His friends came to visit, and for the first time, he couldn’t join in the conversations, couldn’t interject with thoughts and jokes. He sat there, silent. “I love debating and arguing and being heard,” he told me. “And joking – that was really hard. You can’t have much of a wit when you need to write everything down. You miss the moment.”This is one thing you lose when you’re taken outside the flow of conversation. The other, as Joe would discover, is the privilege of being included as an equal. “People treat you differently,” Joe wrote. “They don’t mean to, but they patronise you, treat you like a child.”In the months that have passed since his glossectomy, Joe has made slow and steady progress with his physical therapy. The natural timbre of his voice is lower than it was before the surgery, though it may rise as the swelling continues to go down. “I fear I’ve said my last ‘S’ sound,” he typed to me over the summer. Ls and Js are also difficult, which frustrates him, because he struggles to say his own name, and that of his wife, Louisa.When I spoke to him in late November, he cheerfully reported that his Ss were nearly back, if a little bit reminiscent of Sean Connery. He prefers to get along with his occasionally muddled natural speech, but he has found his digital voice helpful as a reference in his speech and language sessions. Recently, he started a new job at an ad agency and used his VocaliD version to show his new colleagues his “old voice”.Joe may not end up using an AAC device every day, but he insists that it was important to him that, no matter what, his voice exists somewhere. “My wife is a Harry Potter fan, so I joke that this is my Horcrux,” he said, referring to the object in which a wizard can hide a part of his soul, and so attain a kind of immortality. He saw it as an act of self-preservation. “I was worried that as I grew older and this became more and more a past event, I might start to forget the sound of my own voice.”People often use the Voicebank this way, Patel says. Early on, she noticed that a disproportionate number of the people who “banked” their voices were transgender people in some stage of transition, often at the beginning, before starting hormone-replacement therapy. For them, as for Joe, the bank might serve as a vault for safeguarding an old self. The record is there to find, just in case: this is who I was.For others, the digital voice is not a remnant of who they were, but a promise of who they will be. Sara Young got her new voice just before Christmas, in the VocaliD offices. Patel and Meltzner shifted nervously on their feet as they stood before Sara and Amy Young, making smalltalk and cueing up the two voices Meltzner had designed for her to choose between. He played the first, using a sentence he had pre-programmed. “HI, MY NAME IS SARA. I’M 16 YEARS OLD, AND I’M AWESOME.”It sounded tinny and halting, like “Heather’s” younger sister, but with a trace of something idiosyncratic and human at the base of the sound.Sara laughed delightedly. “OK,” Patel said. “Now we’ll hear the second one.”“HI, MY NAME IS SARA. I’M 16 YEARS OLD, AND I’M AWESOME.” The second voice sounded clearer, more bell-like. It seemed older than the first in its assurance, but younger in its vitality.“OK, which do you like?” Patel asked Sara.After a long pause, Sara asked for the second.“Oh, phew!” Patel laughed. “That was our favourite, too. What do you like about number two?”After a long pause, Sara said: “IT’S SPUNKY.”They downloaded it on to her device. Patel pointed out to me later that the moment when a person’s new voice is turned on can be anti-climactic, because they’re not quite sure how to respond to it. The really interesting stuff, she says, comes in the days and weeks after, when the client notices how they’re treated differently, or in watching how they psychologically internalise the experience of having a voice that sounds like them." 

In [144]:
%%time
sc_arcticle = scores_doc_coref1(nlp(doc))

CPU times: user 24.2 s, sys: 3.62 s, total: 27.8 s
Wall time: 30.2 s


In [145]:
def score_sim_arcticle(doc,score_doc,k):
    """Prend en paramètre un doc, le fichier de scores associés à ce doc et un entier k correspondant à l'indice de la 
    description avec laquelle on veut comparer le doc. Retourne le score de similarité entre le doc et la description.
    Ici on calcule le score avec la méthode neuralcoref1 : on utilise liste_scores1.
    Rajout d'une pénalité par entité différentes 
    """
    res=0
    score_image=liste_scores1[k]
    for i in score_doc.keys():
        if i in score_image.keys():
            res += (score_doc[i]+score_image[i])/2 
    score_image_tri = tri_dict(score_image)
    c = 0 
    for i in score_image_tri.keys():
        c+=1
        if i not in score_doc.keys():
            
            res -= 1/(c+1)**2
        
    return res


In [146]:
%%time
best_image11(nlp(doc))

31208

In [147]:
df1.description[31208]

"The late Ravi Shankar's son-in-law Joe Right speaks at his memorial at the Self realization fellowship grounds in Encinitas, California on December 20, 2012. Concerts and private memorials in India and California have been held in memory of the legendary Indian Sitar player who died on December 11, 2012 at the age of 92 in southern California.AFP PHOTO / Frederic J. BROWN"